In [60]:
import sqlite3
import hashlib
import requests
from bs4 import BeautifulSoup
from joblib import Parallel, delayed
import MeCab as mc
import numpy as np
from gensim.models import word2vec

In [13]:
conn = sqlite3.connect('./tabelog_reviews.sqlite3')
cur = conn.cursor()
cur.execute('create table restaurants(id INTEGER PRIMARY KEY AUTOINCREMENT, name TEXT, rate INTEGER, url TEXT)')
cur.execute('create table reviews(id integer, content text, restaurant_id integer, digest BLOB, FOREIGN KEY(restaurant_id) REFERENCES restaurants(id))')

conn.commit()
conn.close()

In [29]:
restaurants, ratings, urls = [],[],[]
for page in range(1,51):
    url = f'https://tabelog.com/rstLst/steak/{page}/?Srt=D&SrtT=rt&sort_mode=1&sk=%E3%82%B9%E3%83%86%E3%83%BC%E3%82%AD&svt=1900&svps=2&select_sort_flg=1'
    html_soup = BeautifulSoup(requests.get(url).text, 'html.parser')
    for item in html_soup.find_all('a', class_='list-rst__rst-name-target cpy-rst-name js-ranking-num'):
        restaurants.append(item.text)
        urls.append(item.get('href'))
    for item in html_soup.find_all('span', class_='c-rating__val c-rating__val--strong list-rst__rating-val'):
        ratings.append(item.text)

In [31]:
conn = sqlite3.connect('./tabelog_reviews.sqlite3')
cur = conn.cursor()

for idx, (name, rate, url) in enumerate(zip(restaurants, ratings, urls)):
    cur.execute('insert into restaurants values (?,?,?,?)', (idx, name, rate, url))

conn.commit()
conn.close()

In [40]:
conn = sqlite3.connect('./tabelog_reviews.sqlite3')
cur = conn.cursor()
cur.execute('select id,url from restaurants')
res = cur.fetchall()
conn.close()

In [48]:
tail = 'dtlrvwlst/?smp=0&lc=2&PG='
rev_results = []

def get_review(review):
    try:
        res = requests.get('https://tabelog.com' + review, timeout=5)
        return BeautifulSoup(res.text, 'html.parser').find('div',class_='rvw-item__rvw-comment').text
    except Exception as e:
        print(e)

In [58]:
conn = sqlite3.connect('./tabelog_reviews.sqlite3')
cur = conn.cursor()

for index, url in res:
    print(index)
    iteration = 1
    while(True):
        url = url + tail + str(iteration)
        try:
            r = requests.get(url,timeout=5)
            if r.status_code != 200:
                break
            iteration += 1
            review_soup = BeautifulSoup(r.text, 'html.parser')
            review_urls =  [item.get('href') for item in review_soup.find_all('a', class_='rvw-simple-item__title-target')]
            
            results = Parallel(n_jobs=4,verbose=10)([
                    delayed(get_review)(i) for i in review_urls
                ])
            for result in results:
                cur.execute('insert into reviews values (null,?,?,?)', (result, index, hashlib.md5(result.encode()).hexdigest()))
                conn.commit()
        except Exception as e:
            print(e)
            conn.commit()
#             conn.close()
#             break
conn.close()

0


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:    3.5s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    4.9s
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:    6.8s
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    9.7s
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:   12.3s
[Parallel(n_jobs=4)]: Done  42 out of  44 | elapsed:   15.3s remaining:    0.7s
[Parallel(n_jobs=4)]: Done  44 out of  44 | elapsed:   15.8s finished


1


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:    2.0s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    3.7s
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:    6.2s
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    8.3s
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:   11.5s
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   14.6s
[Parallel(n_jobs=4)]: Done  53 tasks      | elapsed:   18.5s
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:   21.6s
[Parallel(n_jobs=4)]: Done  82 out of  82 | elapsed:   26.4s finished


2


...

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:    1.7s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    2.9s
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:    4.5s
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    6.4s
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    8.7s
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   10.8s
[Parallel(n_jobs=4)]: Done  53 tasks      | elapsed:   13.5s
[Parallel(n_jobs=4)]: Done  60 out of  60 | elapsed:   14.7s remaining:    0.0s
[Parallel(n_jobs=4)]: Done  60 out of  60 | elapsed:   14.7s finished


998


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:    2.0s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    3.2s
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:    4.8s
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    6.4s
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    8.9s
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   10.5s
[Parallel(n_jobs=4)]: Done  53 tasks      | elapsed:   12.9s
[Parallel(n_jobs=4)]: Done  68 out of  68 | elapsed:   15.7s remaining:    0.0s
[Parallel(n_jobs=4)]: Done  68 out of  68 | elapsed:   15.7s finished


999


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:    2.5s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    3.8s
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:    5.6s
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    7.5s
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    9.9s
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   12.0s
[Parallel(n_jobs=4)]: Done  53 tasks      | elapsed:   14.6s
[Parallel(n_jobs=4)]: Done  61 out of  61 | elapsed:   16.2s remaining:    0.0s
[Parallel(n_jobs=4)]: Done  61 out of  61 | elapsed:   16.2s finished


In [59]:
def mecab_analysis(texts):
    t = mc.Tagger("-Ochasen")
    # 辞書ロードうまくいかない
    #t = mc.Tagger( '-Owakati -d /Users/tominagaryota/mecab-ipadic-neologd')
    t.parse('')
    output = []
    node =  t.parseToNode(texts)  
    while node:
        if node.surface != "":  # ヘッダとフッタを除外
            word_type = node.feature.split(",")[0]
            if word_type in ['名詞', '形容詞']:
                output.append(node.surface)
            #output.append(node.surface)
        node = node.next
        if node is None:
            break
    return output

In [61]:
model = word2vec.Word2Vec.load('./model/wiki.model')

In [62]:
model.most_similar(positive = ['犬'])

/Users/tominagaryota/.anyenv/envs/pyenv/versions/3.6.2/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('猫', 0.8706178665161133),
 ('ウサギ', 0.8539745807647705),
 ('ネコ', 0.818726122379303),
 ('キツネ', 0.817758321762085),
 ('ネズミ', 0.8163456320762634),
 ('子犬', 0.814063549041748),
 ('クマ', 0.8122203350067139),
 ('オオカミ', 0.8110953569412231),
 ('ヤギ', 0.8104976415634155),
 ('柴犬', 0.790104329586029)]

In [64]:
def query(sql):
    conn = sqlite3.connect('./tabelog_reviews.sqlite3')
    cur = conn.cursor()
    try:
        cur.execute(sql)
        res = cur.fetchall()
    except Exception as e:
        print(e)
        res = None
    finally:
        conn.close()
        return res

In [65]:
sql = "select content from reviews"
reviews = query(sql)

In [66]:
len(reviews)

74189

In [74]:
reviews = [review[0].strip('\n').strip(' ').strip("\n") for review in reviews]

In [77]:
wakati_reviews = [mecab_analysis(review) for review in reviews]

In [78]:
wakati_reviews[0]

['うし',
 'ごろ',
 'グループ',
 '最高峰',
 '西麻布',
 '銀座',
 'エス',
 '東京',
 '数',
 '焼肉店',
 '中',
 'トップクラス',
 '肉質',
 '全席',
 '個室',
 '贅沢',
 '空間',
 'こちら',
 '常連',
 '様',
 'ありがたき',
 '予約',
 '福岡',
 'セレブリティ',
 '連れ',
 '目眩',
 'く肉',
 '宴',
 'シック',
 '高級感',
 '個室',
 '空間',
 'ドラム缶',
 '格差',
 '大き',
 '1日',
 '1',
 '組',
 '銀座',
 '店',
 '限定',
 'G',
 'コース',
 '¥20',
 '000',
 'シャンパーニュ',
 '乾杯',
 '菜の花',
 '新じゃが',
 'ワラビ',
 '春',
 '野菜',
 '今宵',
 '肉',
 '塊',
 '登場',
 '惚れ惚れ',
 '美しい',
 '肉',
 '気品',
 '佇まい',
 '序盤',
 '牛刺し',
 '見事',
 '温度',
 '管理',
 'ウニ',
 '一緒',
 'よう',
 '肉',
 '食感',
 'こと',
 '昨今',
 '稀少',
 'ユッケ',
 'まろやか',
 '味わい',
 '奈良漬け',
 'アクセント',
 '良い',
 '牛刺し',
 'ノックアウト',
 'そう',
 '勢い',
 '赤ワイン',
 '幸せ',
 '度',
 'シンプル',
 'タン',
 'すごい',
 '仕上がり',
 'の',
 'タン',
 'サクッ',
 '食感',
 '柔らか',
 'ジューシー',
 '味わい',
 '他',
 '店',
 '格',
 '違い',
 '圧巻',
 'の',
 '場',
 'サーロイン',
 'しゃぶしゃぶ',
 '肉',
 '厚み',
 '温度',
 '計算',
 '1',
 '品',
 '出汁',
 '投入',
 '程度',
 '驚き',
 '柔らか',
 'さ',
 '旨味',
 '肉質',
 '良さ',
 'S',
 '高い',
 '肉',
 '技術',
 'ギリギリ',
 '生',
 'メンチカツ',
 '他',
 '衣',
 '歯ごたえ',
 '感じ',
 '中',
 '肉',
 '柔らか'

sizeはベクトルの次元数
min_countは最低出現回数
sgは学習アルゴリズム指定(1: skig-pram 2: CBOW)

In [ ]:
tabelog_model = word2vec.Word2Vec(wakati_reviews, size=1000,min_count=5,window=5,iter=1000,workers=4, sg=1)